<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/AIT_Hybrid_Models_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================== STEP 1: Mount & Inspect Files ==================
# - Mounts Google Drive
# - Scans common project folders (edit ROOTS if needed)
# - Prints a 2-level tree and categorizes likely parent/quantized/hybrid artifacts

from google.colab import drive
drive.mount('/content/drive')

import os, sys, math, glob
from pathlib import Path
from datetime import datetime

# ---- EDIT if your project lives elsewhere ----
ROOTS = [
    "/content/drive/MyDrive/DiabeticProject",
    "/content/drive/MyDrive/DiabeticProject/saved_models",
    "/content/drive/MyDrive/DiabeticProject/tflite",
    "/content/drive/MyDrive/DiabeticProject/hybrid_models",
]

INTERESTING_EXTS = {".keras", ".h5", ".tflite", ".joblib", ".pkl", ".onnx", ".pt"}

def fmt_bytes(n):
    units = ["B","KB","MB","GB","TB"]
    i = 0
    while n >= 1024 and i < len(units)-1:
        n /= 1024.0; i += 1
    return f"{n:.3f} {units[i]}"

def list_tree(root, depth=2, max_files=50):
    root = Path(root)
    if not root.exists():
        print(f"• {root}  (missing)")
        return
    print(f"• {root}")
    # level 1
    entries = sorted(root.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
    shown = 0
    for e in entries:
        if shown >= max_files:
            print("   └─ … (truncated)")
            break
        if e.is_dir():
            print(f"   └─ {e}")
            if depth >= 2:
                # level 2 inside e
                try:
                    sub = sorted(e.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
                    sub_shown = 0
                    for s in sub:
                        if sub_shown >= 10:
                            print("      └─ …")
                            break
                        tag = "📄" if s.is_file() else "📁"
                        print(f"      └─ {tag} {s.name}")
                        sub_shown += 1
                except Exception as ex:
                    print(f"      └─ [error opening]: {ex}")
        else:
            size = fmt_bytes(e.stat().st_size)
            print(f"   └─ 📄 {e.name}  ({size})")
        shown += 1

def scan_files(roots):
    found = []
    for r in roots:
        for ext in INTERESTING_EXTS:
            for p in glob.glob(os.path.join(r, "**", f"*{ext}"), recursive=True):
                try:
                    st = os.stat(p)
                    found.append({
                        "path": p,
                        "ext": ext.lower(),
                        "size": st.st_size,
                        "size_h": fmt_bytes(st.st_size),
                        "mtime": st.st_mtime,
                        "mtime_h": datetime.fromtimestamp(st.st_mtime).strftime("%Y-%m-%d %H:%M"),
                        "name": os.path.basename(p),
                        "parent": os.path.basename(os.path.dirname(p)),
                    })
                except FileNotFoundError:
                    pass
    return sorted(found, key=lambda x: (-x["size"], x["path"].lower()))

def print_section(title):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))

# ---------- Show trees ----------
print_section("== SAMPLE TREE (2 levels) ==")
for r in ROOTS:
    list_tree(r, depth=2)

# ---------- Gather interesting files ----------
all_files = scan_files(ROOTS)

# Heuristics: categorize likely artifacts (you'll confirm)
parents   = [f for f in all_files if f["ext"] in {".keras", ".h5"}]
quantized = [f for f in all_files if f["ext"] == ".tflite"]
hybrids   = [f for f in all_files if f["ext"] in {".joblib", ".pkl"}]

def show_table(title, items, limit=25):
    print_section(title + f"  (showing up to {limit})")
    if not items:
        print("None found here.")
        return
    for i, f in enumerate(items[:limit], 1):
        print(f"{i:2d}. {f['size_h']:>9}  {f['mtime_h']}  {f['path']}")

show_table("PARENT CANDIDATES (.keras / .h5)", parents)
show_table("QUANTIZED CANDIDATES (.tflite)", quantized)
show_table("HYBRID CANDIDATES (.joblib / .pkl)", hybrids)

# Quick summary counts
print_section("== SUMMARY ==")
print(f"Total interesting files: {len(all_files)}")
print(f"  Parents (.keras/.h5): {len(parents)}")
print(f"  Quantized (.tflite):  {len(quantized)}")
print(f"  Hybrids (.joblib/.pkl): {len(hybrids)}")

print("\n✅ Done. Tell me which paths are your parent, quantized, and hybrid models.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

== SAMPLE TREE (2 levels) ==
• /content/drive/MyDrive/DiabeticProject
   └─ /content/drive/MyDrive/DiabeticProject/hybrid_models
      └─ 📄 DenseNet121_fp16_FE_feature_extractor.keras
      └─ 📄 DenseNet121_fp16_FE_KNN.joblib
      └─ 📄 DenseNet121_fp16_FE_RandomForest.joblib
      └─ 📄 DenseNet121_fp16_FE_SVM_RBF.joblib
      └─ 📄 hybrid_test_metrics.csv
   └─ /content/drive/MyDrive/DiabeticProject/sample_images
   └─ /content/drive/MyDrive/DiabeticProject/saved_models
      └─ 📁 DenseNet121_single
      └─ 📁 EfficientNetV2B0_single
      └─ 📁 NASNetMobile_single
      └─ 📁 ResNet50V2_single
      └─ 📄 DenseNet121_single_split.keras
      └─ 📄 hybrid_size_breakdown.csv
      └─ 📄 hybrid_size_densefp16.csv
      └─ 📄 hybrid_size_replace_allheads.csv
      └─ 📄 inference_timing_hybrid_READY.csv
      └─ 📄 NASNetMobile_single_split.keras
      └─ …
   └─ /cont